In [1]:
import pandas as pd
import numpy as np
import sys
import torch

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_train["ones"] = 1.
xcolnams += ["ones"]
Xs_train = torch.tensor(df_train[xcolnams[1:]].values.T).float().cuda()
print("check2")

check0
check1
check2


In [4]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

In [5]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_test["ones"] = 1.
xcolnams += ["ones"]
Xs_test = torch.tensor(df_test[xcolnams[1:]].values.T).float().cuda()
print("check2")

check0
check1
check2


In [6]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

In [7]:
Ys_train = torch.tensor(df_train[ycolnams + ["ones"]].values.T).float().cuda()

In [8]:
Ys_test = torch.tensor(df_test[ycolnams + ["ones"]].values.T).float().cuda()

In [9]:
sys.path.append(r"C:\Users\maxim\Desktop\js\linfit")
from torch_linfit import lay, lay_holder

In [10]:
Ys_train.shape

torch.Size([11, 60000])

In [11]:
model = lay_holder([lay(784,400), lay(400,10)]) 
#model = lay_holder([lay(784,10, False)])

In [12]:
import timeit
start_time = timeit.default_timer()
NTR = 1
for j in range(NTR):
    print("Back")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=len(model.layers)-i)
        forw = model.forward(Xs_train)
        mse = torch.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

    print("Front")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=i + 1)
        forw = model.forward(Xs_train)
        #print(forw.shape)
        mse = torch.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)
elapsed = timeit.default_timer() - start_time
print(f"Took {elapsed}s")

Back
tensor(0.0220, device='cuda:0')
tensor(0.0431, device='cuda:0')
Front
tensor(0.0449, device='cuda:0')
tensor(0.0103, device='cuda:0')
Took 3.758156099999951s


In [13]:
forw = model.forward(Xs_train)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = torch.argmax(forw[:-1,:], axis=0).cpu().numpy()
# print(mse)

In [14]:
guessed

array([5, 0, 4, ..., 5, 6, 5], dtype=int64)

In [15]:
sum(guessed == df_train["label"].values) / len(guessed)

0.90045

In [16]:
1/10

0.1

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(df_train["label"].values, guessed)

array([[5735,    2,   28,   15,   10,   41,   49,    4,   37,    2],
       [   1, 6477,   61,   21,    9,   17,    8,   19,  119,   10],
       [  66,   62, 5346,  129,   71,   18,   86,   50,  111,   19],
       [  25,   35,  192, 5345,   13,  210,    9,  112,  137,   53],
       [   7,   22,   41,    3, 5382,   22,   75,    9,   44,  237],
       [  88,   24,   69,  237,   33, 4509,   98,   40,  288,   35],
       [  56,   20,   51,    2,   44,  109, 5591,    1,   39,    5],
       [  21,   80,   61,   44,   99,   16,    2, 5730,   17,  195],
       [  39,  235,   87,  199,   74,  262,   86,   16, 4777,   76],
       [  38,   13,   24,   89,  320,   67,   19,  199,   45, 5135]],
      dtype=int64)

In [19]:
forw = model.forward(Xs_test)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = torch.argmax(forw[:-1,:], axis=0).cpu().numpy()
# print(mse)

In [20]:
sum(guessed == df_test["label"].values) / len(guessed)

0.8991

In [21]:
confusion_matrix(df_test["label"].values, guessed)

array([[ 958,    0,    2,    1,    1,    7,    5,    1,    5,    0],
       [   0, 1099,    7,    3,    1,    1,    3,    3,   18,    0],
       [  11,   10,  922,   19,   16,    2,   14,   13,   23,    2],
       [   5,    1,   24,  895,    3,   35,    0,   17,   25,    5],
       [   1,    4,    6,    1,  900,    1,   14,    5,   11,   39],
       [  11,    1,    5,   48,   10,  733,   10,   13,   56,    5],
       [  17,    5,    8,    1,    8,   19,  894,    2,    4,    0],
       [   5,   18,   19,   10,   14,    0,    0,  905,    4,   53],
       [  10,   23,    6,   29,   16,   42,   14,    7,  819,    8],
       [  10,    2,    1,   14,   68,    8,    5,   23,   12,  866]],
      dtype=int64)